Ion simulation

In [ ]:
import numpy as np
from scipy.stats import poisson
import random
import matplotlib.pyplot as plt

In [ ]:
R_B = 20 / 200  #Bright state rate counts per 200us detection time (now per microseconds)
R_D = 0.1 / 200
p_bitflip_success=0.9
ion = CalciumIon(0, R_D, R_B)

photon_bins = np.zeros(100)
trials = 100000
for i in range(trials):
    ion = CalciumIon(0, R_D, R_B)
    ion.update_ion(200, 1)
    n_photons = ion.get_photon_counts()
    photon_bins[int(n_photons)]+=1
    
probab = photon_bins / trials
plt.semilogy(probab)

In [ ]:
def update_likelihood(rho_previous, new_counts, mu):
    """ at step S the likelihood is the product of poissonian distribution"""
    return rho_previous * poisson.pmf(new_counts,mu)

def get_probabilities(rho_D, rho_B):
    """The probability of being bright is calculated with bayes rule."""
    s = rho_D + rho_B
    return (rho_D/s, rho_B/s)

def get_entropy(PD, PB):
    """returns the expected entropy value given the probability distribution"""
    H = -( PD*np.log(PD) + PB*np.log(PB)) 
    return H

def update_ion(current_state, pi_pulse=False, p_bitflip_success=0.99, RD=0.1/200, RB=20/200, dt = 10):
    """Returns the state and photon count at the end of a detection bin period"""
    # if pi pulse is applied, see if successful
    if pi_pulse:
        rand = random.random()
        if rand < p_bitflip_success:
            next_state = 1 - current_state # qubit flip
        else:
            next_state = current_state
    else:
        next_state = current_state
        
    # additional dynamics like leakage
    
        
    # calculate photon counts
    ## get mean
    mu = dt * (RD + next_state * RB) # RB gets added to the dark photon count if state == 1
    n_photons = np.random.poisson(mu)
            
    return next_state, n_photons

def is_pi_pulse_needed(rho_smin_B, rho_smin_D, mu_B, mu_D):
    """rho_smin_B, rho_smin_D are the likelihood at the previous step"""
    H_diff_tot=0
    max_n=10
    for ns in range(0,max_n):
        # calculate the possible likelihoods if we apply or not a pi pulse
        rho_s_B = update_likelihood(rho_smin_B, ns, mu_B)
        rho_s_D = update_likelihood(rho_smin_D, ns, mu_D)
        
        rho_sPi_B = update_likelihood(rho_smin_B, ns, mu_D) # if we were to apply a pi pulse
        rho_sPi_D = update_likelihood(rho_smin_D, ns, mu_B)
        # calculate the associated probabilities
        (P_s_D, P_s_B) = get_probabilities(rho_s_D, rho_s_B)
        (P_sPi_D, P_sPi_B) = get_probabilities(rho_sPi_D, rho_sPi_B)
        
        #calculate the entropies
        H_s = get_entropy(P_s_D, P_s_B)
        H_sPi = get_entropy(P_sPi_D, P_sPi_B)
        
        H_diff_tot =H_diff_tot + (H_sPi- H_s)*(poisson.pmf(ns,mu_B)+poisson.pmf(ns,mu_D))
#         print(H_diff_tot)
    
    if H_diff_tot > 0:
        return True
    else:
        return False
    
def detection(starting_state=0, optimize=True, nbins=10, verbose=False):
    pi_pulse=False
    rho_s_B=1
    rho_s_D=1
    reversed_state=False
    e_c= 0.0001
    current_state = starting_state
    n_pi_pulses=0
    
    bin_time = 200./nbins # in us
    mu_B = R_B *bin_time;
    mu_D = R_D *bin_time;

    for s in range(0,nbins):
        current_state, n_photons = update_ion(current_state, pi_pulse, dt=bin_time)
        counts = int(round(n_photons)); #PB probability of being bright
        if verbose:
            print(counts)
        if reversed_state:
            rho_s_B = update_likelihood(rho_s_B, counts, mu_D)
            rho_s_D = update_likelihood(rho_s_D, counts, mu_B)
        else:
            rho_s_B = update_likelihood(rho_s_B, counts, mu_B)
            rho_s_D = update_likelihood(rho_s_D, counts, mu_D)
        # calculate errors
        eB= rho_s_D/(rho_s_B + rho_s_D)
        eD= rho_s_B/(rho_s_B + rho_s_D)
         
        if min(eB,eD)<e_c:
            bright = 1
            if rho_s_B < rho_s_D:
                bright= 0
            return(s+1, bright, n_pi_pulses)
        else:
            if optimize:
                pi_pulse = is_pi_pulse_needed(rho_s_B, rho_s_D,mu_B, mu_D)
                if(pi_pulse):
                    n_pi_pulses+=1
                    reversed_state= not reversed_state
                    if verbose:
                        print("pi pulse needed")
                    
    bright = 1
    if rho_s_B < rho_s_D:
        bright = 0
    return (nbins+1, bright,n_pi_pulses)
    

In [ ]:
detection(starting_state=0, optimize=True, nbins=20)

In [ ]:
n_trails = 10000
n_success_dark = 0
n_success_bright = 0
n_error_dark = 0
n_error_bright = 0

for i in range(n_trails):
    # choose state
    qubit = round(random.random())
    if qubit == 0:
        bins, predict, n_pi_pulses = detection(starting_state=qubit, optimize=False, nbins=20)
        if predict == qubit:
            n_success_dark+=1
        else:
            n_error_dark+=1
    else:
        bins, predict, n_pi_pulses = detection(starting_state=qubit, optimize=False, nbins=20)
        if predict == qubit:
            n_success_bright+=1
        else:
            n_error_bright+=1

print(n_success_dark / (n_success_dark + n_error_dark), n_success_bright / (n_success_bright + n_error_bright))
    

In [ ]:
stats_sample = {'a': 1, 'b': 2}
stats_tot = {}
for key in stats_sample.keys():
    stats_tot[key+'1'] = np.array([])
    
print(stats_tot)

for i in range(10):
    rand1 = round(random.random() * 5)
    rand2 = round(random.random() * 5)
    stats_i = {'a': rand1, 'b': rand2}
    for key in stats_i.keys():
        stats_tot[key] = np.append(stats_tot[key], stats_i[key])
print(stats_tot)

In [ ]:
print(stats_tot['a'])

In [ ]:
ion1 = CalciumIon()
ion2 = CalciumIon()
ion_list = [ion1, ion1, ion1]
print(ion_list)

In [ ]:
def unique_instances(list_instances):
    unique_instances = []
    for inst in list_instances:
        if inst in unique_instances:
            continue
        else:
            unique_instances.append(inst)
        
    return unique_instances

In [ ]:
unique_inst = unique_instances(ion_list)
print(unique_inst)

In [ ]:
for i in range(20):
    print(i)
    if i == 5:
        break

In [ ]:
from PyonDetect.ion.ion import BerylliumIon, CalciumIon
from PyonDetect.estimator.estimator import EntropyGainEstimator, ThresholdingEstimator, AdaptiveMLEstimator, NonPoissonianAdaptiveMLEstimator
from PyonDetect.environment.environment import SimulationEnvironment

In [ ]:
n_bins = 20
t_detection = 200
tau_db = 1 * 5e2 # in us 
tau_bd = 2 * 1e5 # in us
ion = CalciumIon(initial_state = 1)
estimator = EntropyGainEstimator(R_D = 0.1/200, R_B = 25/200, e_c=5e-4, n_subbins_max=n_bins, n_counts_max=10)
estimator_1 = AdaptiveMLEstimator(R_D = 0.1/200, R_B = 25/200, e_c=5e-4, n_subbins_max=n_bins)
estimator_2 = NonPoissonianAdaptiveMLEstimator(R_D = 0.1/200, R_B = 25/200, e_c=5e-4, tau_db=tau_db, tau_bd=tau_bd, n_subbins_max=n_bins)

env = SimulationEnvironment([ion], [estimator], detection_time=t_detection, n_subbins = n_bins, n_repetition=5000, verbose=True)
env.repeat_single_shots()

In [ ]:
env.save_to_csv()
estimator.get_estimator_type()

In [ ]:
from PyonDetect.ion.ion import BerylliumIon, CalciumIon
from PyonDetect.estimator.estimator import EntropyGainEstimator, ThresholdingEstimator, AdaptiveMLEstimator, NonPoissonianAdaptiveMLEstimator
from PyonDetect.environment.environment import SimulationEnvironment
import numpy as np
import multiprocessing as mp
def run_multiple_environments(ion_list, estimator_list, n_bins, initial_state):
    ion = CalciumIon(initial_state = initial_state)
    estimator = EntropyGainEstimator(R_D = 0.1/200, R_B = 25/200, e_c=5e-4, n_subbins_max=n_bins, n_counts_max=10)
    env = SimulationEnvironment([ion], [estimator], detection_time=200, n_subbins = n_bins, n_repetition=100)
    env.repeat_single_shots()
    return env.get_stats()
    
n_bins = np.arange(3, 21)
initial_state = np.repeat(1, len(n_bins))
print(len(initial_state), len(n_bins))
with mp.Pool(mp.cpu_count() * 2) as p:
    res = p.starmap(run_multiple_environments, zip(n_bins, initial_state))
    


In [ ]:
print(res[17])

In [ ]:
import numpy as np
int(round(np.random.poisson(10)))

In [3]:
import numpy
numpy.finfo('float').eps

2.220446049250313e-16